In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT

from op_fourier_trafo import *
from tools.classical import *
from tools.quantum import *
from jump_ops import sigmam_LCU

In [2]:
num_qubits = 3
eps = 0
bohr_bound = 0

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=False)

Original spectrum:  [-4.     -3.4641 -2.8284 -2.      2.      2.8284  3.4641  4.    ]
Ideal spectrum:  [-0.      0.067   0.1464  0.25    0.75    0.8536  0.933   1.    ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaled coefficients:  [0.125 0.125 0.125 0.125]


In [ ]:
qr_sys = QuantumRegister(num_qubits, 'sys')
qr_b = QuantumRegister(1, 'b')
cr_b = ClassicalRegister(1, 'cb')
circ = QuantumCircuit(qr_sys, qr_b, cr_b)

jump_op = sigmam_LCU()